BeautifulSoup 및 Selenium을 이용한 고파스 크롤링.

'호랑이광장' 게시판의 게시물 및 댓글/대댓글 수집해 csv파일로 작성.

ID/비밀번호 정보는 XXXX로 바꾸어 업로드.

In [0]:
import re
import csv
from selenium import webdriver
from bs4 import BeautifulSoup as bs
driver = webdriver.Chrome('C:\\Users\\goldh\\advEngPhone\\Midterm\\Crawling practice\\chromedriver.exe')
driver.get('https://www.koreapas.com/bbs/main.php')
driver.find_element_by_name('user_id').send_keys('XXXX')
driver.find_element_by_name('password').send_keys('XXXX')
driver.find_element_by_xpath('//*[@class="btn_login btn-small"]').click()
url = 'https://www.koreapas.com/bbs/view.php?id=tiger&no='

In [0]:
#시작하는 게시물 넘버링 설정
num = int(input())

#게시물 5000개 단위로 크롤링 후 저장.
with open('kopas_'+str(num)+'-'+str(int(num)+5000)+'.csv', 'w',encoding='utf-8') as f: # 이거로 파일 열어오면, 나중에 f.close 까먹어도 사단 날 일이 없다.
    writer = csv.writer(f)
    writer.writerow(['main', 'reply_num', 'connection', 'reply'])
    for i in range(330000,335001):
        url_use = url+str(i)
        driver.get(url_use)
        html = driver.page_source
        bsobject = bs(html, 'html.parser')
         
        try:
            text_body = bsobject.find("div",itemprop="articleBody").get_text().strip() #본문 획득
            replies = bsobject.find_all("td", align="left", style=['word-break:break-word;padding-top:4px;font-size:15px;', "word-break:break-word;padding-top:4px;font-size:18px;"] )
            # 댓글 획득
            if replies == []:
                continue
            sorted_replies = [] 
            # 댓글 중 신고 파트 및 앞 뒤 \n 제거
            writer.writerow([text_body,'0','',''])
            for reply in replies:
                unwanted = reply.find('span')
                unwanted.extract()
                sorted_replies.append(reply.get_text().strip())
            for line in sorted_replies:
                reply_num = str(sorted_replies.index(line)+1)
                connection = {'0/'}
                try: # 텍스트 없는 댓글 생략
                    connection_reply = re.findall('(\d+)/',line)
                    connection = set(connection_reply)
                    line = re.sub('\d/','',line)                  
                    
                except:
                    pass
                writer.writerow([i, reply_num, connection, line])
        except:
            continue
    f.close()